In [1]:
# import libaries
import numpy as np
import pandas as pd
import nltk, pprint
import matplotlib.pyplot as plt
import random

import gzip, os, pickle # gzip for reading the gz files, pickle to save/dump trained model 
import _pickle as cPickle

import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# supress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
filename = 'atis.fold0.pkl.gz'
f = gzip.open(filename, 'rb')
try:
    train_set, valid_set, test_set, dicts = pickle.load(f, encoding='latin1')
except:
    train_set, valid_set, test_set, dicts = pickle.load(f)
finally:
    f.close()

In [3]:
print(type(train_set))
print()

# types of the three elements in the tuple
print(type(train_set[0]), type(train_set[1]), type(train_set[2]))
print(len(train_set[0]), len(train_set[1]), len(train_set[2]))

<class 'tuple'>

<class 'list'> <class 'list'> <class 'list'>
3983 3983 3983


In [4]:
print(type(valid_set[0]), type(valid_set[1]), type(valid_set[2]))
print(len(valid_set[0]), len(valid_set[1]), len(valid_set[2]))
print()

# test set
print(type(test_set[0]), type(test_set[1]), type(test_set[2]))
print(len(test_set[0]), len(test_set[1]), len(test_set[2]))

<class 'list'> <class 'list'> <class 'list'>
995 995 995

<class 'list'> <class 'list'> <class 'list'>
893 893 893


In [5]:
pprint.pprint(train_set[0][:3])
print('#'*50)
pprint.pprint(train_set[1][:3])
print('#'*50)
pprint.pprint(train_set[2][:3])

[array([554, 194, 268,  64,  62,  16,   8, 234, 481,  20,  40,  58, 234,
       415, 205]),
 array([554, 241, 481,  14, 200,  91,  26, 239]),
 array([232,   0, 273, 502, 254, 481, 165, 193, 208,  77, 502,  64])]
##################################################
[array([  0,   0,   0,  18,   0,   1,  52,   0,   0,  76,   0,   0,   0,
        18, 109]),
 array([  0,   0,   0,   0,   0,   6, 107, 107]),
 array([ 0,  0,  0,  0,  0,  0, 44,  0,  0, 18,  0, 18])]
##################################################
[array([126, 126, 126,  48, 126,  36,  35, 126, 126,  33, 126, 126, 126,
        78, 123]),
 array([126, 126, 126, 126, 126,   2,  83,  83]),
 array([126, 126, 126, 126, 126, 126,  42, 126, 126,  48, 126,  78])]


In [6]:
train_x, _, train_label = train_set
val_x, _, val_label = valid_set
test_x, _, test_label = test_set

In [7]:
train_x[0]

array([554, 194, 268,  64,  62,  16,   8, 234, 481,  20,  40,  58, 234,
       415, 205])

In [8]:
train_label[0]

array([126, 126, 126,  48, 126,  36,  35, 126, 126,  33, 126, 126, 126,
        78, 123])

In [9]:
print(type(dicts))
print(dicts.keys())

<class 'dict'>
dict_keys(['labels2idx', 'tables2idx', 'words2idx'])


In [10]:
print(type(dicts['labels2idx']))
print(type(dicts['tables2idx']))
print(type(dicts['words2idx']))

<class 'dict'>
<class 'dict'>
<class 'dict'>


In [11]:
words = dicts['words2idx']
labels = dicts['labels2idx']
tables = dicts['tables2idx']

In [12]:
random.sample(words.items(), 10)

[('seventh', 435),
 ('same', 414),
 ('plane', 381),
 ('takeoffs', 469),
 ('quebec', 392),
 ('sixteen', 441),
 ('buy', 84),
 ('f', 179),
 ('pittsburgh', 379),
 ('c', 87)]

In [13]:
[k for val in train_x[0] for k,v in words.items() if v==val]

['what',
 'flights',
 'leave',
 'atlanta',
 'at',
 'about',
 'DIGIT',
 'in',
 'the',
 'afternoon',
 'and',
 'arrive',
 'in',
 'san',
 'francisco']

In [14]:
sents = []
for i in range(30):
    sents.append(' '.join([k for val in train_x[i] for k,v in words.items() if v==val]))

sents

['what flights leave atlanta at about DIGIT in the afternoon and arrive in san francisco',
 'what is the abbreviation for canadian airlines international',
 "i 'd like to know the earliest flight from boston to atlanta",
 'show me the us air flights from atlanta to boston',
 'show me the cheapest round trips from dallas to baltimore',
 "i 'd like to see all flights from denver to philadelphia",
 'explain fare code qx',
 "i 'd like a united airlines flight on wednesday from san francisco to boston",
 'what is the price of american airlines flight DIGITDIGIT from new york to los angeles',
 'what does the meal code s stand for',
 'what are all flights to denver from philadelphia on sunday',
 'what times does the late afternoon flight leave from washington for denver',
 'what flights are available monday from san francisco to pittsburgh',
 'what airlines have business class',
 'flights from atlanta to washington dc',
 'from new york to toronto on thursday morning',
 'show me all the direct

In [15]:
random.sample(labels.items(), 25)

[('I-meal_code', 111),
 ('B-toloc.country_name', 79),
 ('B-arrive_date.month_name', 8),
 ('I-arrive_date.day_number', 85),
 ('B-depart_time.period_of_day', 33),
 ('B-depart_time.end_time', 31),
 ('B-class_type', 18),
 ('B-cost_relative', 21),
 ('B-depart_date.year', 30),
 ('B-days_code', 24),
 ('I-stoploc.city_name', 119),
 ('B-depart_date.month_name', 28),
 ('I-fare_amount', 102),
 ('I-arrive_time.end_time', 86),
 ('B-toloc.state_name', 81),
 ('B-stoploc.state_code', 72),
 ('B-return_time.period_of_day', 65),
 ('B-fare_amount', 38),
 ('B-day_number', 23),
 ('B-economy', 37),
 ('B-fare_basis_code', 39),
 ('B-arrive_date.today_relative', 9),
 ('B-day_name', 22),
 ('I-fare_basis_code', 103),
 ('B-flight', 40)]

In [16]:
print(len(labels.keys()))

127


In [17]:
id_to_words = {words[k]:k for k in words}
id_to_labels = {labels[k]:k for k in labels}

In [18]:
def print_query(index):
    w = [id_to_words[id] for id in train_x[index]]
    l = [id_to_labels[id] for id in train_label[index]]
    return list(zip(w, l))

In [19]:
print_query(3900)

[('please', 'O'),
 ('show', 'O'),
 ('me', 'O'),
 ('the', 'O'),
 ('return', 'O'),
 ('flight', 'O'),
 ('number', 'O'),
 ('from', 'O'),
 ('toronto', 'B-fromloc.city_name'),
 ('to', 'O'),
 ('st.', 'B-toloc.city_name'),
 ('petersburg', 'I-toloc.city_name')]

In [20]:
print_query(3443)

[('is', 'O'),
 ('there', 'O'),
 ('a', 'O'),
 ('flight', 'O'),
 ('between', 'O'),
 ('oakland', 'B-fromloc.city_name'),
 ('and', 'O'),
 ('boston', 'B-toloc.city_name'),
 ('with', 'O'),
 ('a', 'O'),
 ('stopover', 'O'),
 ('in', 'O'),
 ('dallas', 'B-stoploc.city_name'),
 ('fort', 'I-stoploc.city_name'),
 ('worth', 'I-stoploc.city_name'),
 ('on', 'O'),
 ('twa', 'B-airline_code')]

In [21]:
i=random.randrange(len(train_x))
print_query(i)

[('list', 'O'),
 ('the', 'O'),
 ('flights', 'O'),
 ('arriving', 'O'),
 ('in', 'O'),
 ('atlanta', 'B-toloc.city_name'),
 ('from', 'O'),
 ('boston', 'B-fromloc.city_name'),
 ('between', 'O'),
 ('DIGIT', 'B-arrive_time.start_time'),
 ('and', 'O'),
 ('DIGIT', 'B-arrive_time.end_time'),
 ('pm', 'I-arrive_time.end_time')]

In [22]:
i=random.randrange(len(train_x))
print_query(i)

[('saturday', 'B-depart_date.day_name'),
 ('flights', 'O'),
 ('from', 'O'),
 ('las', 'B-fromloc.city_name'),
 ('vegas', 'I-fromloc.city_name'),
 ('to', 'O'),
 ('montreal', 'B-toloc.city_name')]

In [25]:
def pos_tag(sent_list):
    pos_tags = []    
    for sent in sent_list:
        tagged_words = nltk.pos_tag([id_to_words[val] for val in sent])
        pos_tags.append(tagged_words)
    return pos_tags

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\avinash.tiwari\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [26]:
train_pos = pos_tag(train_x)
valid_pos = pos_tag(val_x)
test_pos = pos_tag(test_x)

In [27]:
i = random.randrange(len(train_pos))
train_pos[i]

[('flights', 'NNS'),
 ('from', 'IN'),
 ('montreal', 'NN'),
 ('and', 'CC'),
 ('phoenix', 'NN'),
 ('to', 'TO'),
 ('las', 'VB'),
 ('vegas', 'NN'),
 ('arriving', 'VBG'),
 ('at', 'IN'),
 ('the', 'DT'),
 ('same', 'JJ'),
 ('time', 'NN')]